# Blockchain and Crypto Economics - Liquidity pools

## Set-up

In [1]:
%%capture
pip install web3

In [2]:
# Import the Web3 object and establish an HTTP connection
# See also: https://medium.com/validitylabs/
# how-to-interact-with-the-ethereum-blockchain-and-create-a-database-with-python-and-sql-3dcbd579b3c0

import web3
from web3 import Web3
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Dropdown, BoundedFloatText
import math
import getpass
import warnings
warnings.filterwarnings('ignore')

# Connecting to the UZHETH Blockchain
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
# Connection test
print(web3.isConnected())

True


In [3]:
# Connecting to deployed smart contracts
RouterAddress, FactoryAddress = "0x043124a8e838FfdFB968309Fe4077B274d5A4C34", "0x07Db82eAd90449bf347fc3A4fAAE32F86eCC8B60"

router = web3.eth.contract(address=RouterAddress, abi=open('./Router.abi', 'r').read())
factory = web3.eth.contract(address=FactoryAddress, abi=open('./Factory.abi', 'r').read())

### Helper funtions

In [4]:
# Account selection helper
nullAddress = "0x0000000000000000000000000000000000000000"
selectedAccount = nullAddress
userDropdown = Dropdown(description = "Select account:", options = web3.eth.accounts)

def handle_change():
    pass

def selectAccount():
    userDropdown.on_trait_change(handle_change, name="value")
    display(userDropdown)

In [39]:
# Token selection helper
pools, tokens = {}, {}

def updateTokenDropdown(tokens):
    tokenDropdown1 = Dropdown(description = "Select token 1:", options = [(tokens[i][0], i) for i in tokens.keys()])
    tokenDropdown2 = Dropdown(description = "Select token 2:", options = [(tokens[i][0], i) for i in tokens.keys()])
    
    return tokenDropdown1, tokenDropdown2

def setUpTokens(pools, tokens):
    for i in range(0, factory.functions.countPools().call()):
        pool = web3.eth.contract(address=factory.functions.allPools(i).call(), abi=open('./Pool.abi', 'r').read())
        pools[pool.address] = pool
        token0 = web3.eth.contract(address=pool.functions.token0().call(), abi=open('./ERC20.abi', 'r').read())
        tokens[token0.address] = (token0.functions.symbol().call(), token0)
        token1 = web3.eth.contract(address=pool.functions.token1().call(), abi=open('./ERC20.abi', 'r').read())
        tokens[token1.address] = (token1.functions.symbol().call(), token1)
    tokenDropdown1, tokenDropdown2 = updateTokenDropdown(tokens)
    
    return pools, tokens, tokenDropdown1, tokenDropdown2
    
def selectToken():
    tokenDropdown1.on_trait_change(handle_change, name="value")
    tokenDropdown2.on_trait_change(handle_change, name="value")
    display(tokenDropdown1, tokenDropdown2)
    
def addToken(pools, tokens):
    newToken = str(input("Enter token address:"))
    if newToken != "": 
        token = web3.eth.contract(address=newToken, abi=open('./ERC20.abi', 'r').read())
        tokens[token.address] = (token.functions.symbol().call(), token)
        print("Token {} successfully added".format(tokens[token.address][0]))
    tokenDropdown1, tokenDropdown2 = updateTokenDropdown(tokens)
    return pools, tokens, tokenDropdown1, tokenDropdown2

def confirmTokenSelection(tokenA, tokenB):
    if tokenA == tokenB: 
        print("Tokens cannot be the same")
        return nullAddress, nullAddress, nullAddress
    else:
        print("Token selection confirmed for", tokenA, "and", tokenB)
        return tokenA, tokenB, factory.functions.getPool(tokenA, tokenB).call()

def getPoolInfo(tokenA, tokenB):
    pool = factory.functions.getPool(tokenA, tokenB).call()
    if pool == nullAddress: 
        print("No pool available")
    else:
        reserves = pools[pool].functions.getReserves().call()
        if tokenA < tokenB: 
            reservesA, reservesB = reserves[0], reserves[1]
        else:
            reservesA, reservesB = reserves[1], reserves[0]
        liquidity = pools[pool].functions.totalSupply().call()
        print("Overview of corresponding liquidity pool")
        print("Current amount of {} in pool: {}".format(tokens[tokenA][0], reservesA/(10**18)))
        print("Current amount of {} in pool: {}".format(tokens[tokenB][0], reservesB/(10**18)))
        print("Current amount of liquidity tokens in circulation:", liquidity/(10**18))
    
def getTokenAmounts(tokenA, tokenB):
    print("Overview of balances of selected account")
    print("Current amount of {}: {}".format(tokens[tokenA][0], tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18)))
    print("Current amount of {}: {}".format(tokens[tokenB][0], tokens[tokenB][1].functions.balanceOf(selectedAccount).call()/(10**18)))
    if pool == nullAddress: pass
    else: print("Current amount of liquidity tokens: {}".format(pools[pool].functions.balanceOf(selectedAccount).call()/(10**18)))
    
pools, tokens, tokenDropdown1, tokenDropdown2 = setUpTokens(pools, tokens)

In [31]:
# Transactions helper
def unlockAccount(account = str(web3.eth.default_account)):
    print("Unlocking account {}".format(account))
    return web3.geth.personal.unlock_account(account, getpass.getpass())
        
def lockAccount(account = str(web3.eth.default_account)):
    print("Locking account {}".format(account))
    return web3.geth.personal.lock_account(account)
    
def approveAmount(token, amount, recipient=router.address, unlocked=False):
    if not unlocked: print("Account locked")
    if token in tokens:
        txApproval = tokens[token][1].functions.approve(recipient, math.ceil(amount*(10**18))).transact({'from': selectedAccount})
        txApprovalReceipt = web3.eth.wait_for_transaction_receipt(txApproval)
        print("{} {} approved".format(tokens[token][1].functions.allowance(selectedAccount, recipient).call()/(10**18), tokens[token][0]))
    elif token in pools:
        txApproval = pools[token].functions.approve(recipient, math.ceil(amount*(10**18))).transact({'from': selectedAccount})
        txApprovalReceipt = web3.eth.wait_for_transaction_receipt(txApproval)
        print("{} liquidity tokens approved".format(pools[token].functions.allowance(selectedAccount, recipient).call()/(10**18)))

In [7]:
# Exchange helper
def setUpExchange(pool, tokens):
    caption = widgets.Label(value="Exchange {} for {}".format(tokens[tokenA][0], tokens[tokenB][0]))
    amountIn = BoundedFloatText(value=1.0, min=0.0, max=tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18), 
                                 step=0.000000000000000001, description="Amount in")
    amountOut = BoundedFloatText(value=router.functions.getQuote(tokenA, tokenB, math.ceil(amountIn.value*(10**18))).call()[0]/(10**18),
                                 min=0.0, max=tokens[tokenA][1].functions.balanceOf(pool).call()/(10**18), 
                                 step=0.000000000000000001, description="Amount out", disabled=True)
    return caption, amountIn, amountOut

def on_changeExchange(change):
    amountOut.value = router.functions.getQuote(tokenA, tokenB, math.ceil(amountIn.value*(10**18))).call()[0]/(10**18)

def interactExchange():
    amountIn.observe(on_changeExchange, names='value')
    display(caption, amountIn, amountOut)
    
def exchangeTokens(tokenIn, amountIn, tokenOut, amountOutMin=0, unlocked=False):
    if not unlocked: print("Account locked")
    else:
        beforeBalance = (tokens[tokenIn][1].functions.balanceOf(selectedAccount).call()/(10**18), tokens[tokenOut][1].functions.balanceOf(selectedAccount).call()/(10**18))
        txExchange = router.functions.exchangeTokens(tokenIn, tokenOut, math.ceil(amountIn*(10**18)), amountOutMin).transact({'from': selectedAccount})
        txExchangeReceipt = web3.eth.wait_for_transaction_receipt(txExchange)
        afterBalance = (tokens[tokenIn][1].functions.balanceOf(selectedAccount).call()/(10**18), tokens[tokenOut][1].functions.balanceOf(selectedAccount).call()/(10**18))
        print("Transaction completed: {} {} exchanged for {} {}".format(tokens[tokenIn][0], (beforeBalance[0]-afterBalance[0])/(10**18), tokens[tokenOut][0], (afterBalance[1]-beforeBalance[1])/(10**18)))

In [41]:
# Liquidity helper
def setUpLiquidity(pool, tokens):
    caption1 = widgets.Label(value="Set amount of {} and {} to add to liquidity pool".format(tokens[tokenA][0], tokens[tokenB][0]))
    amountA = BoundedFloatText(value=0.0, min=0.0, max=tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18), 
                                 step=0.000000000000000001, description="Amount {}".format(tokens[tokenA][0]))
    amountB = BoundedFloatText(value=0.0, min=0.0, max=tokens[tokenB][1].functions.balanceOf(selectedAccount).call()/(10**18), 
                                 step=0.000000000000000001, description="Amount {}".format(tokens[tokenB][0]))
    caption2 = widgets.Label(value="Or set amount of liquidity tokens to send back to pool")
    amountLiquidity = BoundedFloatText(value=0.0, min=0.0, max=pools[pool].functions.balanceOf(selectedAccount).call()/(10**18), 
                                       step=0.000000000000000001, description="Amount liquidity tokens")
    return caption1, caption2, amountA, amountB, amountLiquidity

def on_changeAddLiquidity(change):
    pass

def on_changeWithdrawLiquidity(change):
    pass

def interactLiquidity():
    amountA.observe(on_changeAddLiquidity, names='value')
    amountB.observe(on_changeAddLiquidity, names='value')
    amountLiquidity.observe(on_changeWithdrawLiquidity, names='value')
    display(caption1, amountA, amountB)
    display(caption2, amountLiquidity)

def addLiquidity(tokenA, amountA, tokenB, amountB, pools, unlocked=False):
    if not unlocked: print("Account locked")
    else:
        pool = factory.functions.getPool(tokenA, tokenB).call()
        beforeBalance = [tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18), tokens[tokenB][1].functions.balanceOf(selectedAccount).call()/(10**18)]
        if len(pools.keys()) > 0: beforeBalance.append(pools[pool].functions.balanceOf(selectedAccount).call()/(10**18))
        else: beforeBalance.append(0)
        txAdd = router.functions.addLiquidity(tokenA, tokenB, math.ceil(amountA*(10**18)), math.ceil(amountB*(10**18))).transact({'from': selectedAccount})
        txAddReceipt = web3.eth.wait_for_transaction_receipt(txAdd)
        if len(pools.keys()) < 0: 
            pool = web3.eth.contract(address=factory.functions.getPool(tokenA, tokenB).call(), abi=open('Pool.abi', 'r').read())
            pools[pool.address] = pool
            pool = pool.address
        afterBalance = [tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18), tokens[tokenB][1].functions.balanceOf(selectedAccount).call()/(10**18), pools[pool].functions.balanceOf(selectedAccount).call()/(10**18)]
        print("Transaction completed: {} {} and {} {} added to pool for {} liquidity tokens".format(tokens[tokenA][0], (beforeBalance[0]-afterBalance[0])/(10**18), tokens[tokenB][0], (beforeBalance[1]-afterBalance[1])/(10**18), (afterBalance[2]-beforeBalance[2])/(10**18)))
    return pools
        
def withdrawLiquidity(pool, amountLiquidity, unlocked=False):
    if not unlocked: print("Account locked")
    else:
        beforeBalance = (tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18), tokens[tokenB][1].functions.balanceOf(selectedAccount).call()/(10**18), pools[pool].functions.balanceOf(selectedAccount).call()/(10**18))
        txWithdraw = router.functions.withdrawLiquidity(tokenA, tokenB, math.ceil(amountLiquidity*(10**18))).transact({'from': selectedAccount})
        txWithdrawReceipt = web3.eth.wait_for_transaction_receipt(txWithdraw)
        afterBalance = (tokens[tokenA][1].functions.balanceOf(selectedAccount).call()/(10**18), tokens[tokenB][1].functions.balanceOf(selectedAccount).call()/(10**18), pools[pool].functions.balanceOf(selectedAccount).call()/(10**18))
        print("Transaction completed: {} liquidity tokens returned to pool for {} {} and {} {}".format((beforeBalance[2]-afterBalance[2])/(10**18), tokens[tokenA][0], (afterBalance[0]-beforeBalance[0])/(10**18), tokens[tokenB][0], (afterBalance[1]-beforeBalance[1])/(10**18)))

## Interacting

### Select account
Select an account for transactions

*Select a local account from dropdown*

In [9]:
selectedAccount = selectAccount()

Dropdown(description='Select account:', options=('0x2fF965D61cD315369AAA2CfED6506bd1da40248B', '0x5C5eF49707C8…

*Execute to confirm account selection*

In [12]:
selectedAccount = userDropdown.value
web3.eth.default_account = selectedAccount
print("Account selection confirmed for", selectedAccount)
print("Available balance in UZHETH: {}".format(web3.fromWei(web3.eth.getBalance(selectedAccount), "ether")))

Account selection confirmed for 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5
Available balance in UZHETH: 9.994481229


### Select tokens
Select a pair of ERC20 tokens and get information on the corresponding liquidity pool and balances of the selected account.

*Execute to add ERC20 token to selection (need to add liquidity before exchanging tokens)*

In [19]:
pools, tokens, tokenDropdown1, tokenDropdown2 = addToken(pools, tokens)

Enter token address:


*Select ERC20 tokens from dropdown*

In [45]:
pools, tokens, tokenDropdown1, tokenDropdown2 = setUpTokens(pools, tokens)
selectToken()

Dropdown(description='Select token 1:', options=(('TT3', '0x128A8F1a0361adA3ab1F6Bb0F4940461B3BD3f1E'), ('TT4'…

Dropdown(description='Select token 2:', options=(('TT3', '0x128A8F1a0361adA3ab1F6Bb0F4940461B3BD3f1E'), ('TT4'…

*Execute to confirm token selection*

In [46]:
tokenA, tokenB = tokenDropdown1.value, tokenDropdown2.value
tokenA, tokenB, pool = confirmTokenSelection(tokenA, tokenB)

Token selection confirmed for 0x1D596687e91C0Deb82aa2538858b4FBD0331141A and 0x128A8F1a0361adA3ab1F6Bb0F4940461B3BD3f1E


*Get information on selected liquidity pool*

In [53]:
getPoolInfo(tokenA, tokenB)

Overview of corresponding liquidity pool
Current amount of TT4 in pool: 29.807505117973523
Current amount of TT3 in pool: 32.47012615100116
Current amount of liquidity tokens in circulation: 31.46402724563618


*Get information on currently owned amounts of tokens*

In [54]:
getTokenAmounts(tokenA, tokenB)

Overview of balances of selected account
Current amount of TT4: 18.49811350057503
Current amount of TT3: 14.801904500339539
Current amount of liquidity tokens: 31.46402724563618


### Exchange tokens
Exchange the first selected token against the second selected token using liquidity.

*Input token amount for exchange*

In [24]:
if len(pools.keys()) > 0:
    caption, amountIn, amountOut = setUpExchange(pool, tokens)
    interactExchange()
else: print("No pool available. Add liquidity first.")

Label(value='Exchange TT3 for TT4')

BoundedFloatText(value=1.0, description='Amount in', max=7.272030651340702, step=1e-18)

BoundedFloatText(value=1.0, description='Amount out', disabled=True, max=40.0, step=1e-18)

*Execute to exchange tokens*

In [49]:
unlocked = unlockAccount()
approveAmount(tokenA, amountIn.value, unlocked=unlocked)
exchangeTokens(tokenA, amountIn.value, tokenB, unlocked=unlocked)
locked = lockAccount()

Unlocking account 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5
········
3.0 TT4 approved
Transaction completed: TT4 3e-18 exchanged for TT3 3.857142857142857e-18
Locking account 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5


### Manage liquidity
Provide liquidity for the selected token pair in return for liquidity tokens or withdraw provided liquidity.

*Input amounts of tokens to add or amount of liquidity tokens to withdraw from pool*

In [42]:
caption1, caption2, amountA, amountB, amountLiquidity = setUpLiquidity(pool, tokens)
interactLiquidity()

Label(value='Set amount of TT3 and TT4 to add to liquidity pool')

BoundedFloatText(value=0.0, description='Amount TT3', max=5.625, step=1e-18)

BoundedFloatText(value=0.0, description='Amount TT4', max=15.913483667505783, step=1e-18)

Label(value='Or set amount of liquidity tokens to send back to pool')

BoundedFloatText(value=0.0, description='Amount liquidity tokens', max=37.01958280119174, step=1e-18)

*Execute to add liquidity*

In [43]:
unlocked = unlockAccount()
approveAmount(tokenA, amountA.value, unlocked=unlocked)
approveAmount(tokenB, amountB.value, unlocked=unlocked)
pools = addLiquidity(tokenA, amountA.value, tokenB, amountB.value, pools, unlocked=unlocked)
locked = lockAccount()

Unlocking account 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5
········
5.0 TT3 approved
10.0 TT4 approved
Transaction completed: TT3 5e-18 and TT4 3.888888888888889e-18 added to pool for 4.444444444444443e-18 liquidity tokens
Locking account 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5


*Execute to withdraw liquidity*

In [52]:
unlocked = unlockAccount()
approveAmount(pool, amountLiquidity.value, unlocked=unlocked)
withdrawLiquidity(pool, amountLiquidity.value, unlocked=unlocked)
locked = lockAccount()

Unlocking account 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5
········
10.0 liquidity tokens approved
Transaction completed: 1e-17 liquidity tokens returned to pool for TT4 9.473518721958136e-18 and TT3 1.031976164319668e-17
Locking account 0x5C5eF49707C82ca38DB4877D0Db3E13859e308b5


## Deploying
Admin functions for deploying and setting up new instances of the router and factory contracts.

In [210]:
# Enter admin adress (needed for deployment and initialisation of factory and router contracts)
admin = str(input("Enter admin address:"))

# Deploy new instance of factory contract
unlockAccount(admin)
newRouter = web3.eth.contract(abi=open('./Router.abi', 'r').read(), bytecode=open('./Router.bin', 'r').read())
txDeployRouter = newRouter.constructor(admin).transact({'from': admin})
txDeployRouterReceipt = web3.eth.wait_for_transaction_receipt(txDeployRouter)
router = web3.eth.contract(address=txDeployRouterReceipt.contractAddress, abi=open('./Router.abi', 'r').read())
print("New router deployed at {}".format(router.address))

# Deploy new instance of router contract
newFactory = web3.eth.contract(abi=open('./Factory.abi', 'r').read(), bytecode=open('./Factory.bin', 'r').read())
txDeployFactory = newFactory.constructor(admin).transact({'from': admin})
txDeployFactoryReceipt = web3.eth.wait_for_transaction_receipt(txDeployFactory)
factory = web3.eth.contract(address=txDeployFactoryReceipt.contractAddress, abi=open('./Factory.abi', 'r').read())
print("New factory deployed at {}".format(factory.address))

# Link factory and router contracts
txSetRouter = factory.functions.setRouter(router.address).transact({'from': admin})
txSetRouterReceipt = web3.eth.wait_for_transaction_receipt(txSetRouter)
txSetFactory = router.functions.setFactory(factory.address).transact({'from': admin})
txSetFactory = web3.eth.wait_for_transaction_receipt(txSetFactory)
lockAccount(admin)

# RouterAddress and FactoryAddress variables under 'Set-up' should be changed to start up with new instances next time

Enter admin address:0xF5c978Fa5c02D2a13C1Fb2dDEAf1FbA6a44eA677
Unlocking account 0xF5c978Fa5c02D2a13C1Fb2dDEAf1FbA6a44eA677
········
New router deployed at 0x043124a8e838FfdFB968309Fe4077B274d5A4C34
New factory deployed at 0x07Db82eAd90449bf347fc3A4fAAE32F86eCC8B60
Locking account 0xF5c978Fa5c02D2a13C1Fb2dDEAf1FbA6a44eA677


True